In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [10]:
dataset = pd.read_csv(r"C:\Users\vinny\Downloads\loan_approval_dataset.csv")
dataset.head()

,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,1,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved
1,2,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected
2,3,3,Graduate,No,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected
3,4,3,Graduate,No,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected
4,5,5,Not Graduate,Yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected


In [11]:
dataset.shape # it checks the size of my data set

(4269, 13)

In [12]:
dataset.nunique() # it tells how many unique values exist in each column

loan_id                      4269
 no_of_dependents               6
 education                      2
 self_employed                  2
 income_annum                  98
 loan_amount                  378
 loan_term                     10
 cibil_score                  601
 residential_assets_value     278
 commercial_assets_value      188
 luxury_assets_value          379
 bank_asset_value             146
 loan_status                    2
dtype: int64

In [14]:
#duplicates
dataset.duplicated() # The dataset.duplicated() method in pandas checks for duplication in out dataset. It only returns a boolean series(True for 
# True for duplicate rows and false for unique rows

0       False
1       False
2       False
3       False
4       False
        ...  
4264    False
4265    False
4266    False
4267    False
4268    False
Length: 4269, dtype: bool

In [15]:
dataset.duplicated().sum() # the sum method counts  the number of duplicates 

np.int64(0)

In [16]:
dataset.columns # The dataset.columns command in pandas returns the names of all columns in the dataset.


Index(['loan_id', ' no_of_dependents', ' education', ' self_employed',
       ' income_annum', ' loan_amount', ' loan_term', ' cibil_score',
       ' residential_assets_value', ' commercial_assets_value',
       ' luxury_assets_value', ' bank_asset_value', ' loan_status'],
      dtype='object')

In [18]:

dataset.columns = [c.strip() for c in dataset.columns] # This removes extra spaces from column names in the dataset.
dataset.columns

Index(['loan_id', 'no_of_dependents', 'education', 'self_employed',
       'income_annum', 'loan_amount', 'loan_term', 'cibil_score',
       'residential_assets_value', 'commercial_assets_value',
       'luxury_assets_value', 'bank_asset_value', 'loan_status'],
      dtype='object')

In [19]:
# Why X and y are used:
# X:
# In machine learning, X typically represents the input features (the independent variables). These are the columns of the dataset that we will use to predict something.

# y:
# y usually represents the target variable (the dependent variable). This is the column you are trying to predict or model based on the input features.

# The reason why X is often capitalized is to conventionally denote that it contains the input data, typically a 2D matrix (rows and columns). y, on the other hand, is lowercase because it usually represents a single target variable (often a 1D vector).

X = dataset.drop(columns=["loan_status"]) 
y = dataset['loan_status']

In [ ]:
# What does X.info() do?

# When you call X.info(), it displays a summary of the DataFrame X. Specifically, it shows:

# - The number of rows and columns in the DataFrame.
# - The names of the columns.
# - The data type of each column.
# - The number of non-null values (i.e., how many missing values there are in each column).

# So, X.info() does not show the actual values of the data. Instead, it gives you an overview of the structure of the dataset.


In [20]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4269 entries, 0 to 4268
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   loan_id                   4269 non-null   int64 
 1   no_of_dependents          4269 non-null   int64 
 2   education                 4269 non-null   object
 3   self_employed             4269 non-null   object
 4   income_annum              4269 non-null   int64 
 5   loan_amount               4269 non-null   int64 
 6   loan_term                 4269 non-null   int64 
 7   cibil_score               4269 non-null   int64 
 8   residential_assets_value  4269 non-null   int64 
 9   commercial_assets_value   4269 non-null   int64 
 10  luxury_assets_value       4269 non-null   int64 
 11  bank_asset_value          4269 non-null   int64 
dtypes: int64(10), object(2)
memory usage: 400.3+ KB


In [ ]:
# Apply the domain knowledge as part of preprocessing

# Purpose: In some datasets, different types of assets are recorded in separate columns, 
# but they represent different components of a total asset value. To create a new feature, 
# we can combine those asset columns into a single 'total_assets_value' column, which might 
# be more meaningful and useful for analysis or modeling.
#
# Here, we are summing up four different columns that represent different asset values:
# 'residential_assets_value', 'commercial_assets_value', 'luxury_assets_value', and 'bank_asset_value'.
# The resulting sum will be stored in a new column called 'total_assets_value'.
#X['total_assets_value'] = X['residential_assets_value'] + X['commercial_assets_value'] + X['luxury_assets_value'] + X['bank_asset_value']

# Drop the individual asset columns after creating the 'total_assets_value' column.
# Purpose: Once the 'total_assets_value' column is created, we no longer need the original individual asset columns.
# Dropping them will make the dataset cleaner and easier to work with, reducing redundancy.
# Also, if you are using machine learning models, dropping irrelevant features can improve performance.
#
# We are dropping the following columns:
# - 'residential_assets_value'
# - 'commercial_assets_value'
# - 'luxury_assets_value'
# - 'bank_asset_value'
# 
# After this operation, only the 'total_assets_value' column will remain in the dataset, 
# which represents the sum of all asset types.
# 'inplace=True' means the changes will be applied directly to the DataFrame (X) without creating a new variable.
#X.drop(columns=['residential_assets_value', 'commercial_assets_value', 'luxury_assets_value', 'bank_asset_value'], inplace=True)


In [23]:
# Apply the domain knowledge as part of preprocessing
X['total_assets_value'] = X['residential_assets_value'] + X['commercial_assets_value'] + X['luxury_assets_value'] + X['bank_asset_value']

# Drop the Columns
X.drop(columns=['residential_assets_value', 'commercial_assets_value', 'luxury_assets_value', 'bank_asset_value'], inplace=True)
